In [ ]:
# default_exp metric

# Metric

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export

from fastcore.basics import GetAttr, store_attr
from types import FunctionType
from fastai.metrics import Metric, AvgMetric, AccumMetric


class _LearnerProxy(GetAttr):
    _default = 'learn'
    def __init__(self, learn, idx):
        store_attr()
        self.pred = self.learn.pred[idx]
        self.y = self.learn.y[idx]


class RoutedAccumMetric(AccumMetric, GetAttr):
    "AccumMetric with predictions and targets for a specific model head."
    _default = 'metric'
    def __init__(self, idx, metric):
        self.idx = idx
        self.metric = metric
        self._name = metric.name
    
    def reset(self):
        "Clear all targs and preds"
        return self.metric.reset()
    
    def accumulate(self, learn):
        "Store targs and preds from `learn`, using activation function and argmax as appropriate"
        return self.metric.accumulate(_LearnerProxy(learn, self.idx))
    
    def __call__(self, preds, targs):
        "Calculate metric on one batch of data"
        return self.metric(preds[self.idx], targs[self.idx])
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

In [ ]:
#export

def route_to_metric(idx, metric):
    """Routes model output at idx to metric"""
    if isinstance(metric, type): 
        metric = metric()
    if isinstance(metric, FunctionType):
        func = lambda preds, *targs, **kwargs: metric(preds[idx], targs[idx], **kwargs)
        func.__name__ = metric.__name__
        return AvgMetric(func)
    if isinstance(metric, Metric):
        return RoutedAccumMetric(idx, metric)
    raise ValueError("Unsupported metric type; must be either function or Metric")


In [ ]:
#hide

import torch
from fastai.metrics import accuracy, R2Score, F1Score

bs = 8
target1, pred1 = torch.randint(0, 5, [bs], dtype=float), torch.randint(0, 5, [bs], dtype=float)
target2, pred2 = torch.randn(bs), torch.randn(bs)

preds = [pred1, pred2]
targets = [target1, target2]

inner_f1_macro = F1Score(average='macro')
f1_macro = route_to_metric(0, inner_f1_macro)
test_close(f1_macro(preds, targets), inner_f1_macro(preds[0], targets[0]))

inner_r2 = R2Score()
r2 = route_to_metric(1, inner_r2)
test_close(r2(preds, targets), inner_r2(preds[1], targets[1]))

In [ ]:
#export

def mtl_metrics(*metrics_list):
    """Convenience function to route each prediction to list of metrics by their order."""
    return [route_to_metric(i, m) for i, metrics in enumerate(metrics_list) for m in metrics]

In [ ]:
#hide
routed_metrics = mtl_metrics([F1Score(average='macro'), accuracy], [R2Score])
test_eq(3, len(routed_metrics))

Example usage for routing the first model prediction to `F1Score` and `accuracy`, while routing the second one to `R2Score`.
```py
learn = Learner(...
    metrics = mtl_metrics([F1Score(average='macro'), accuracy], [R2Score])
)
```